In [1]:
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
from pyscf import lib
from pyscf.prop.nmr import rhf as rhf_nmr

angstrom = 1 / 0.52917721067

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV.reshape((1,dV.shape[0],dV.shape[1]))


def alchemy_pol_deriv(polobj, with_cphf=True):
    mf = polobj._scf
    mol = mf.mol
    mo_energy = mf.mo_energy
    mo_coeff = mf.mo_coeff
    mo_occ = mf.mo_occ
    occidx = mo_occ > 0
    orbo = mo_coeff[:, occidx]
    orbv = mo_coeff[:,~occidx]

    charges = mol.atom_charges()
    coords  = mol.atom_coords()
    charge_center = np.einsum('i,ix->x', charges, coords) / charges.sum()
    int_r=DeltaV(mol,[.001,-.001])    ########   .001 as finite difference intervall  
    h1 = lib.einsum('xpq,pi,qj->xij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals?
    print(h1.shape, "shape (1, n ao, nocc)" )
    s1 = np.zeros_like(h1)
    vind = polobj.gen_vind(mf, mo_coeff, mo_occ)
    if with_cphf:
        mo1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1, polobj.max_cycle_cphf, polobj.conv_tol)[0]
    else:
        mo1 = rhf_nmr._solve_mo1_uncoupled(mo_energy, mo_occ, h1, s1)[0]
    return mo1


In [2]:
mol=gto.M(atom="C 0 0 0; O 0 0 1.8",unit="Bohr")
mf=scf.RHF(mol)
mf.scf()

converged SCF energy = -111.064463936466


-111.064463936466

In [3]:
plo=mf.Polarizability()
pa=alchemy_pol_deriv(plo)
nao=mol.nao
nocc=mol.nelectron//2 #RHF
U=np.zeros((nao,nao))
U[:,:nocc]=pa[0,:,:nocc]
U=U.T-U
O=np.diag(mf.mo_occ)
C=mf.mo_coeff
dP_app=C@(U@O-O@U)@C.T 

(1, 10, 7) shape (1, n ao, nocc)


In [4]:
dP_app

array([[-1.45140749e-05,  1.19213484e-04,  1.05187102e-20,
        -2.48294511e-19,  1.38005778e-04,  2.17511230e-06,
        -1.27091614e-04,  3.18197023e-21,  1.30759524e-19,
         7.25255489e-05],
       [ 1.19213484e-04,  8.26696074e-05, -1.12380836e-19,
         1.01553968e-18, -1.42278183e-04,  1.58531498e-04,
        -2.59219647e-04, -4.06380739e-19, -2.90860407e-19,
         3.38458869e-04],
       [ 1.05187102e-20, -1.12380836e-19, -5.27823939e-04,
        -1.40946282e-18, -6.74095818e-19,  6.41404971e-20,
         3.78452214e-19, -2.49889522e-04,  7.72494048e-19,
         6.59346858e-20],
       [-2.48294511e-19,  1.01553968e-18, -1.38235777e-18,
        -5.27823939e-04, -2.01946267e-19,  3.76234001e-21,
        -6.35386379e-19,  5.69206141e-19, -2.49889522e-04,
         1.11267399e-19],
       [ 1.38005778e-04, -1.42278183e-04, -6.74095818e-19,
        -2.01946267e-19, -3.86864231e-04,  1.43987876e-04,
        -8.03202793e-06, -8.86967341e-20,  1.38957824e-19,
         1.

In [5]:
from alch_deriv import alc_deriv

In [6]:
alc_deriv(mf)

(1, 10, 7) shape (1, n ao, nocc)


(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -1.01374897e-20, -1.25761268e-20,
          2.69469221e-05],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.82518712e-20, -2.67784413e-20,
          2.24697166e-05],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -1.09333869e-19,  1.47148201e-19,
         -1.36078589e-04],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -2.84814077e-18,  4.00059974e-18,
          5.47797862e-05],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  2.72641406e-04,  1.440791